<a href="https://colab.research.google.com/github/SoleromYess/Simulaci-n-II/blob/main/Pr%C3%A1cticas/Practica_1_(Uniformidad_e_Independencia).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctica 1**
En este notebook se implementarán las **Pruebas de Uniformidad** y las **Pruebas de Independencia** de números aleatorios vistas en clase.



In [ ]:
#Importamos la bibliotecas que se implementarán

import numpy as np  #utilizada principalmente para trabajar con matrices numéricas
import math #para realizar operaciones matemáticas comunes
import random  #generar números pseudoaleatorios
from scipy import integrate #Para realizar integrales
import scipy.stats as st #Para obtner los valores de la distribución normal (z)
import scipy.stats as stats #Para obtner los valores de chi cuadrada
from scipy.stats import kstwobign
from scipy.stats import kstest #Para obtener los valores críticos de Kolmogorov-Smirnov

Se definen algunas funciones para validar los datos ingresados

In [ ]:
#Validar números enteros
def validar_numero(n):
  if n.isnumeric():
      n = int(n)
      return True
  else:
      return False
#======================================================

Se podrán ingresar datos uno a uno o generarlos aleatoriamente

In [ ]:
print("¿Ingresará los datos o prefiere que sean generados aleatoriamente?\n1-Ingresar\n2-Aleatorios");
opc= input();
A = []; #Se crea una lista vacía para los n valores

while opc=="1" or opc=="2":
  if opc == "1":#opción de ingresar datos
    print("Ingresará los datos uno a uno:");

    #Número de datos que se ingresarán

    while True: #Ciclo para validar que la variable sea una entrada válida llamando a la función "validar número"
      n = input("\nIngrese la cantidad de valores a evaluar: "); #leer la variable n
      if n.isnumeric(): # si n es un valor numérico se convertirá a entero
        n = int(n)
        break #terminar el ciclo
      else: #si no es un valor numérico se mostrará el mensaje y se repetirá el ciclo
        print("El valor ingresado no es un número. Intente de nuevo.")

    #Se leerán los datos, uno a uno:

    for i in range(0,n): #ciclo para ingresar los valores uno a uno
      print("Ingrese el dato ",i+1,":" )
      while True: #ciclo que válida que el el valor ingresado sea numérico
        d = input()
        if d.replace('.', '', 1).isdigit():#se usa .isdigit para no rechazar valores flotantes
          d = float(d);
          A.append(d); #ya que el valor es numerico se añade a la lista datos
          break
        else:
          print("El valor ingresado no es un número. Intente de nuevo."); 

    #Ya con los valores ingresados se revisará que estos sean correctos

    while True:
      print("\nLos datos ingresados son los siguientes: ")
      for i in range(0,n):
        print(i+1, ": ",A[i])
      print("\n¿Deseas corregir algún valor?\n1-SI\n2-NO")
      opc = input()
      if opc == "1":#opción de corrección
        error = int(input("\nIngrese el número del valor que desea cambiar: "))#se ingresa el índice del error
        A[error-1] = float(input("\nIntroduzca el nuevo valor: ")) #se ingresa la corrección
      elif opc == "2":
        print("\n¡Perfecto ^u^!\n")
        break
      else:
        print("Opción no válida. Por favor, seleccione '1' o '2'.");

    break

  elif opc == "2": #opción de generar datos aleatorios

    #Número de datos que se generarán

    while True: #Ciclo para validar que la variable sea una entrada válida llamando a la función "validar número"
      n = input("\nIngrese la cantidad de valores a generar: "); #leer la variable n
      if n.isnumeric(): # si n es un valor numérico se convertirá a entero
        n = int(n)
        break #terminar el ciclo
      else: #si no es un valor numérico se mostrará el mensaje y se repetirá el ciclo
        print("El valor ingresado no es un número. Intente de nuevo.")

    #Número de decimales para los aleatorios

    while True: #Ciclo para validar que la variable sea una entrada válida llamando a la función "validar número"
      decimales = input("\nNúmero de decimales que tendrán los valores aleatorios : "); #leer la variable n
      if decimales.isnumeric(): # si n es un valor numérico se convertirá a entero
        decimales = int(decimales)
        break #terminar el ciclo
      else: #si no es un valor numérico se mostrará el mensaje y se repetirá el ciclo
        print("El valor ingresado no es un número. Intente de nuevo.")

  #Límite inferior para los aleatorios

    while True: #Ciclo para validar que la variable sea una entrada válida llamando a la función "validar número"
      inf = input("\nValor mínimo de los números aleatorios: "); #leer la variable n
      if inf.isnumeric(): # si n es un valor numérico se convertirá a entero
        inf = int(inf)
        break #terminar el ciclo
      else: #si no es un valor numérico se mostrará el mensaje y se repetirá el ciclo
        print("El valor ingresado no es un número. Intente de nuevo.")

  #Límite superior para los aleatorios

    while True: #Ciclo para validar que la variable sea una entrada válida llamando a la función "validar número"
      may = input("\nValor mayor de los números aleatorios: "); #leer la variable n
      if may.isnumeric(): # si n es un valor numérico se convertirá a entero
        may = int(may)
        break #terminar el ciclo
      else: #si no es un valor numérico se mostrará el mensaje y se repetirá el ciclo
        print("El valor ingresado no es un número. Intente de nuevo.")
    print("\n")
    #generación de números aleatorios
    for i in range(n):
      A.append(round(random.uniform(inf, may), decimales))
      print(A[i])  

    break
  else:
      print("Opción no válida. Por favor, seleccione '1' o '2'.")

**Prueba de uniformidad**

La prueba de uniformidad es una prueba estadística que se utiliza para verificar si una muestra de datos proviene de una distribución uniforme o no. La distribución uniforme es aquella en la que todos los valores posibles tienen la misma probabilidad de ocurrir, es decir, la distribución es "uniforme" en el rango de valores posibles.

Se generan las hipótesis siguientes para verificar si el grupo de números cumplen con la prueba de uniformidad:

$H_0: r_i \sim U(0,1)$

$H_i: r_i $ no son uniformes

La metodología es la **prueba de bondad de ajuste** $\chi^2$:

1. Se colocan los datos en una tabla de frecuencias de: $m = \sqrt n $   intervalos.

In [ ]:
m=math.sqrt(n) #número de clases o intervalos
#condicionales para aproximar m al número entero más cercano
if m- (m//1) < .5 :
  m=int(m//1)
if m - (m//1) >= .5 :
  m=int(m//1 + 1 )
print(m)

3


Se obtiene la Frecuencia Observada $FO_i$ en cada intervalo $i$.

Se propone una distribución de probabilidad de acuerdo con la forma de la tabla de frecuencias obtenidas. $$F(x) = \int_{l_i}^{l_s} \frac{1}{b-a}dx = \frac{l_s-l_i}{b-a}$$

Luego, calculamos la Frecuencia Esperada para cada uno de los intervalos $(FE_i)$ mediante la integreación de la distribución propuesta y su posterior multiplicación por el número total de datos.

Se calcula el estimador: $$C = \sum_{i=1}^m \frac{(FE_i - FO_i)^2}{FE_i}$$


In [ ]:
A_ord=np.sort(A) #Ordenamos los datos
decimales=int(input("¿a cuántos decimales desea redondear?: "))  #a cuánto se desea redondear
dato_menor = A_ord[0]
dato_mayor = A_ord[n-1]

rango = dato_mayor - dato_menor

Amplitud = round(rango/m , decimales) 

pre_calculo = round(dato_menor+(m*(Amplitud)),1);
exceso= round(pre_calculo-dato_mayor,1);
#print("Amplitud = ",Amplitud)
#print("pre_calculo = ",pre_calculo)
#print("exceso = ",exceso)

if exceso % 2 == 0:
    lim_inf=dato_menor - (exceso/2)
else:
    print("El exceso es impar, el usuario debe hallar dos numeros consecutivos de tal forma que su suma sea igual a ",exceso, "luego,se selecciona el más pequeño de estos:")
    while True: #ciclo que válida que el el valor ingresado sea numérico
      imp=input("ingrese ese valor: ")
      if imp.replace('.', '', 1).isdigit():#se usa .isdigit para no rechazar valores flotantes
        imp = float(imp);
        break
      else:
        print("El valor ingresado no es un número. Intente de nuevo.");

    lim_inf=dato_menor - imp

print("\n      CLASES                  FOi                         F(x)                FEi = F(x)*n   ")
lim_sup=0
j=0
T_frec=0
c=0
for i in range (0,m):
  print(lim_inf, "a < " , end='')
  lim_sup=round(lim_inf+Amplitud,decimales)
  #Aqui se hace la integral para la función de densidad========================================
  def f(x):
    return 1/(dato_mayor-dato_menor)
  fx, error = integrate.quad(f, lim_inf, lim_sup)
  #=============================================================================================
  #Frecuencia observada
  FO=0;
  for i in range (0,n-1):
    if lim_inf<= A_ord[j] <=lim_sup:
      FO=FO+1;
      j=j+1;
    if j==n:
      break

  T_frec=T_frec+FO; #Suma las frecuencias de cada intervalo
  #=============================================================================================
  FE = fx*n #Frecuencia esperada
  c = c+ ((FE-FO)**2/FE)#estimador
  lim_inf=lim_sup
  print(lim_sup,"                ", end='')
  print(FO,"             ", end='')
  print(fx,"             ", end='')
  print(FE, "\n")
print("Total de datos = ",T_frec)

print("\nCalculando el estimador, obtenemos: \nC = ",c)


Si $C \leq x^2$ correspondiente con **$m-k-1$ grados de libertad**, (donde $k =$ número de parámetros estimados de la distribución) y a un nivel de confiabilidad de $1-α$, entonces no se puede rechazar la hipótesis de que los datos siguen la distribución propuesta.


In [ ]:
while True: #Ciclo para validar que la variable sea una entrada válida llamando a la función "validar número"
  k=input("ingrese el número de parámetros estimados de la distribución: "); #leer la variable k
  if k.isnumeric(): # si n es un valor numérico se convertirá a entero
    k = int(k)
    break #terminar el ciclo
  else: #si no es un valor numérico se mostrará el mensaje y se repetirá el ciclo
    print("El valor ingresado no es un número. Intente de nuevo.")

grados_de_libertad = m-k-1;

while True: #ciclo que válida que el el valor ingresado sea numérico
  nivel_de_aceptacion = input("ingrese el nivel de aceptación (por ejemplo: .95 para un 95%): ")
  if nivel_de_aceptacion.replace('.', '', 1).isdigit():#se usa .isdigit para no rechazar valores flotantes
    nivel_de_aceptacion = float(nivel_de_aceptacion);
    break
  else:
    print("El valor ingresado no es un número. Intente de nuevo.");

#alpha= 1-nivel_de_aceptacion

chi_cuad = stats.chi2.ppf(nivel_de_aceptacion, grados_de_libertad)

print("X^2 (\u03B1,grados de libertad)= ",chi_cuad)
per = nivel_de_aceptacion*100
print("\n Finalmente, dado que:", end="")
if c<=chi_cuad:
  print("c<=chi_cuad \nse ACEPTA la hipótesis nula, es decir, los ",n, "números son estadísticamente uniformes con un nivel de aceptación del ", per,"%.")
if c>chi_cuad:
  print("no se cumple c>chi_cuad \nse RECHAZA la hipótesis nula, es decir, los ",n, "datos no son estadísticamente uniformes con un nivel de aceptación del ", per,"%.")

**Prueba de independencia**

Demuestra que los números generados son estadísticamente independientes entre sí. Se proponen las siguientes hipótesis:

$H_0 : r_i \sim $ independiente

$H_i : r_i \sim $ dependiente

Para verificar las hipótesis se realiza una **Prueba de corridas**:
Se clasifica cada número aleatorio con respecto al anterior, de acuerdo con:

Si $r_i \leq r_{i-1}      \Longrightarrow r_i = - = 0$

Si $r_i > r_{i-1}      \Longrightarrow r_i = + = 1$

In [ ]:
r = [];
for i in range (1, n): #ciclo para evaluar cada dato con respecto al anterior
  if A[i] <= A[i-1]:
    r.append(0) #valor negativo
  if A[i] > A[i-1]:
    r.append(1) #valor positivo

print("A = ",A)
print("r = ",r)

A =  [0.97, 0.11, 0.65, 0.26, 0.98, 0.03, 0.13, 0.89, 0.21, 0.69]
r =  [0, 1, 0, 1, 0, 1, 1, 0, 1]


Se calculan las corridas observadas $h$. Una corrida se forma por un conjunto de números aleatorios consecutivos del mismo signo, en este caso:

In [ ]:
h=1 #Se inicializa en 1 porque el ciclo cuenta a partir del segundo signo diferente que se localice
for i in range (1, n-1):
  #print(i)
  if r[i-1] != r[i]:
    #print( ": r[",i,"-1] != r[",i,"]")
    h=h+1;
    
print("h = ", h)

h =  8


Se calcula $E(h)$ y $V(h)$:
$$E(h) = \frac{2n-1}{3}$$

$$V(h) = \frac{16n-29}{90}$$

In [ ]:
E = ((2*n)-1)/(3);
V = ((16*n)-29)/(90);
print("E(h) = (2n-1)/3 = ", "(2*",n,"-1)/3 = ",E)
print("V(h) = (16n-29)/90 = ", "(16*",n,"-29)/90 = ",V)

Se calcula el estadístico $$z = \frac{h-E(h)}{\sqrt{v(h)}}$$
si es menor que el valor crítico $Z_{\frac{\alpha}{2}}$ se acepta la hipótesis de nula.

In [ ]:
z=(h-E)/(math.sqrt(V));
print("z=(h-E(h))/(sqrt(V(h)))= ",z,end = "\n\n")

alpha= 1-nivel_de_aceptacion
nivel_significancia = alpha/2

valor_critico = (-1)*st.norm.ppf(nivel_significancia)

print("Z (\u03B1/2) = ",valor_critico)

print("\nDado que:", end="")
if z<valor_critico:
  print("z<Z(\u03B1/2) \nse ACEPTA la hipótesis nula, es decir, los ",n, "números son estadísticamente independientes con un nivel de aceptación del ", per,"%.")
if z>= valor_critico:
  print("no se cumple z>=Z(\u03B1/2) \nse RECHAZA la hipótesis nula, es decir, los ",n, "datos no son estadísticamente independientes con un nivel de aceptación del ", per,"%.")

**Prueba de uniformidad Kolmogorov-Smirnov**

*Es recomendable para conjuntos $r_i$ pequeños $n<30$.*

Primero, se ordenan de menor a mayor los números del conjunto $r_i$

In [ ]:
print(A_ord)

Se determinan los valores de: $D^+$, $D^-$, y $D$ con las siguientes ecuaciones:

$$D^+ = max_{1<i<n} \{\frac{i}{n}-r_i\}$$

$$D^- = max_{1<i<n} \{r_i - \frac{i-1}{n}\}$$

$$D = max(D^+, D^-)$$

In [ ]:
D_plus = []
D_minus = []

for i in range (0,n): #Ciclo que hará las operaciones por cada i y las guardará en un array 
  #R = A_ord[i]
  j=i+1
  D_plus.append( (j/n)-A_ord[i])
  D_minus.append( A_ord[i]-((j-1)/n))
  
#Dado que ya estan los array, solo se imprmirá el mayor en cada caso
print("D+ = ", max(D_plus))
print("D- = ", max(D_minus))
D=max(max(D_plus),max(D_minus))
print("D = ",D)

D+ =  0.24
D- =  0.19000000000000006
D =  0.24


Se determina el valor crítico $D_{\alpha,n}$ de la tabla de valores críticos de Kolmogorov-Smirnov.

In [ ]:
ks_critico = kstwobign.isf(alpha/2)/np.sqrt(n)
print("valor crítico = ", ks_critico)

Finalmente, si $D > D_{\alpha, n}$ se concluye que el conjunto $r_i$ no sigue una distribución uniforme; de lo contrario se acepta que si la sigue.

In [ ]:
print(D,">",ks_critico)
if D >ks_critico:
   print("El conjunto de números no sigue una distribución uniforme")
if D <=ks_critico:
   print("El conjunto de números sigue una distribución uniforme")